In [1]:
from google.cloud import bigquery
import gc
import pandas as pd
import numpy as np
import datetime
import os
import json
import glob
from haversine import haversine
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/docs/merkle-gdyr-prod-9783607c77a1.json"

zip_centers=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))
client = bigquery.Client()

print(datetime.datetime.now())
print(os.getcwd())

2020-12-21 17:58:33.945299
/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Post_Campaign_Analysis


# Audience Report

In [2]:
df_31_green=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections/GY_green_store_matching_pair_BenJL_20201103.xlsx",sheet_name="Sheet1",dtype=str)
df_9_single=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections/GY_twelve_matching_pair_BenJL_20201103.xlsx",sheet_name="Sheet1",dtype=str)
df_31_green=df_31_green[df_31_green['final_selection']=="1"]
df_9_single=df_9_single[df_9_single['final_selection']=="1"]

# 1
list_test_store_str_id_part_1=[str(int(x)).zfill(4) for x in df_31_green['green_store_id'].tolist()]
list_control_store_str_id_part_1=[str(int(x)).zfill(4) for x in df_31_green['matching_store_id'].tolist()]
# 2
list_test_store_str_id_part_2=[str(int(x)).zfill(4) for x in df_9_single['twelve_store_id'].tolist()]
list_control_store_str_id_part_2=[str(int(x)).zfill(4) for x in df_9_single['matching_store_id'].tolist()]

print(len(list_test_store_str_id_part_1),len(list_control_store_str_id_part_1))
print(len(list_test_store_str_id_part_2),len(list_control_store_str_id_part_2))

print(len(set(list_test_store_str_id_part_1+list_control_store_str_id_part_1+list_test_store_str_id_part_2+list_control_store_str_id_part_2)))

31 31
9 9
80


In [3]:
list_40_test_stores=list_test_store_str_id_part_1+list_test_store_str_id_part_2
list_40_control_stores=list_control_store_str_id_part_1+list_control_store_str_id_part_2
print(len(list_40_test_stores),len(list_40_test_stores))

40 40


In [4]:
print(datetime.datetime.now())
query_crm="""
select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd, t2.state_province_cd, t2.country_cd from mdb.indiv_summary as t1
join mdb.addr as t2 on
t1.cr_addr_key=t2.cr_addr_key
"""

print(query_crm)

df_all_ids=client.query(query_crm).result().to_dataframe()
print(datetime.datetime.now())
print(df_all_ids.shape,df_all_ids['indiv_key'].nunique())

2020-12-21 17:58:34.042553

select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd, t2.state_province_cd, t2.country_cd from mdb.indiv_summary as t1
join mdb.addr as t2 on
t1.cr_addr_key=t2.cr_addr_key



/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


2020-12-21 18:04:48.638623
(7617622, 6) 7617622


## Load Store list

In [5]:
final_uploaded_folder="/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections/output_cleaned_audience_2020-11-06/"
list_uploaded_main_aud_files=glob.glob(final_uploaded_folder+"*.csv")

list_audience_national=[x for x in list_uploaded_main_aud_files if "balance" in x.lower() and "tireandservice" in x.lower()]
list_audience_national.sort()
# list_audience_national

list_audience_40_stores=[x for x in list_uploaded_main_aud_files if "_stores_" in x.lower() and "tireandservice" in x.lower() and "_ecom" in x.lower()]
list_audience_40_stores.sort()
# list_audience_40_stores

## Load segment ids

In [6]:
df_id_segment_label=pd.DataFrame()

for file in list_audience_national+list_audience_40_stores:
    df=pd.read_csv(file,usecols=['indiv_key'])
    df['segment']=os.path.basename(file)
    df_id_segment_label=df_id_segment_label.append(df)
print(df_id_segment_label.shape,df_id_segment_label['indiv_key'].nunique())

(284729, 2) 284489


In [7]:
df=df_id_segment_label.groupby('indiv_key')['segment'].count().to_frame().reset_index()
df=df.sort_values("segment",ascending=False)
df_dup=df[df['segment']>1]
df_dup.shape

(240, 2)

In [8]:
for i in range(len(df_dup)):
    df=df_id_segment_label[df_id_segment_label['indiv_key']==df_dup['indiv_key'].tolist()[i]]
    df=df[df['segment'].str.contains("Test")]
    if len(df)>0:
        print(i,df['segment'].tolist())
# 240 ids are in the control for both 31 and 9, all 40 anyway

In [9]:
df_id_segment_label['segment']=df_id_segment_label['segment'].str.replace("_31_","_40_")
df_id_segment_label['segment']=df_id_segment_label['segment'].str.replace("_9_","_40_")
df_id_segment_label=df_id_segment_label.drop_duplicates()

print(df_id_segment_label.shape,df_id_segment_label['indiv_key'].nunique())
df_id_group_count=df_id_segment_label.groupby("segment")['indiv_key'].count().to_frame().reset_index()
df_id_group_count=df_id_group_count.rename(columns={"indiv_key":"total_ids"})

(284489, 2) 284489


## Tire And Service

In [10]:
str_start_date="'2020-11-16'"
str_end_date="'2020-12-02'"

In [11]:
# Service & Tire & null
print(datetime.datetime.now())
query_both_by_id="""
            select indiv_key, min(invoice_date) as min_date, max(invoice_date) as max_date, 
            count(distinct(case when store_id='ECOM' then trn_id end)) as Ecomm_Trans,
            sum(case when store_id='ECOM' then line_item_total_amt else 0 end) as Ecomm_Revenue,
            count(distinct(case when store_id!='ECOM' then trn_id end)) as Store_Trans,
            sum(case when store_id!='ECOM' then line_item_total_amt else 0 end) as Store_Revenue,
            count(distinct trn_id) as Total_Trans,
            sum(line_item_total_amt) as Total_Revenue
            from mdb.trn_detail
            where indiv_key is not null and invoice_date between %s and %s
            group by indiv_key
            """  %(str_start_date,str_end_date)
print(query_both_by_id)
df_both_by_id=client.query(query_both_by_id).result().to_dataframe()
print(df_both_by_id.shape,df_both_by_id['indiv_key'].nunique())
print(datetime.datetime.now())

2020-12-21 18:04:52.102370

            select indiv_key, min(invoice_date) as min_date, max(invoice_date) as max_date, 
            count(distinct(case when store_id='ECOM' then trn_id end)) as Ecomm_Trans,
            sum(case when store_id='ECOM' then line_item_total_amt else 0 end) as Ecomm_Revenue,
            count(distinct(case when store_id!='ECOM' then trn_id end)) as Store_Trans,
            sum(case when store_id!='ECOM' then line_item_total_amt else 0 end) as Store_Revenue,
            count(distinct trn_id) as Total_Trans,
            sum(line_item_total_amt) as Total_Revenue
            from mdb.trn_detail
            where indiv_key is not null and invoice_date between '2020-11-16' and '2020-12-02'
            group by indiv_key
            
(114609, 9) 114609
2020-12-21 18:05:03.905006


In [12]:
df_both_by_id['ecom_shopper']=np.where(df_both_by_id['Ecomm_Trans']>0,1,0)
df_both_by_id['store_shopper']=np.where(df_both_by_id['Ecomm_Trans']>0,0,1)

In [13]:
df_both_by_id=pd.merge(df_both_by_id,df_id_segment_label,on="indiv_key",how="left")
df_both_by_id['segment']=df_both_by_id['segment'].fillna("balance")

In [14]:
df_both_by_id.head(2)

,indiv_key,min_date,max_date,Ecomm_Trans,Ecomm_Revenue,Store_Trans,Store_Revenue,Total_Trans,Total_Revenue,ecom_shopper,store_shopper,segment
0,13499890992000001,2020-11-22 11:51:40,2020-11-22 11:51:40,1,584.3,0,0.0,1,584.3,1,0,balance
1,6039751716000002,2020-11-30 21:27:08,2020-11-30 21:27:08,1,872.0,0,0.0,1,872.0,1,0,segment_Test_TireAndService_Balance_Ecommerce_...


In [15]:
print(df_both_by_id['min_date'].min(),df_both_by_id['min_date'].max())
print(df_both_by_id['max_date'].min(),df_both_by_id['max_date'].max())
# df_both_by_id[df_both_by_id['min_date']!=df_both_by_id['max_date']]

2020-11-16 00:00:00 2020-12-02 00:00:00
2020-11-16 00:00:00 2020-12-02 00:00:00


In [16]:
func_agg={"indiv_key":"count","ecom_shopper":"sum","store_shopper":"sum",
          "Ecomm_Trans":"sum","Ecomm_Revenue":"sum",
          "Store_Trans":"sum","Store_Revenue":"sum",
          "Total_Trans":"sum","Total_Revenue":"sum",
          "min_date":"min","max_date":"max"
         }

df_output_audience_both=df_both_by_id.groupby('segment').agg(func_agg).reset_index()
df_output_audience_both=df_output_audience_both.rename(columns={"indiv_key":"shoppers"})
df_output_audience_both=pd.merge(df_id_group_count,df_output_audience_both,on="segment",how="outer")
df_output_audience_both=df_output_audience_both.sort_values("segment")

In [17]:
df_both_by_id.shape

(114609, 12)

In [18]:
df_both_by_id[df_both_by_id['Total_Revenue']<=0.0001].shape

(21065, 12)

In [19]:
df_both_by_id[df_both_by_id['Total_Revenue']<=0.0001].head(2)

,indiv_key,min_date,max_date,Ecomm_Trans,Ecomm_Revenue,Store_Trans,Store_Revenue,Total_Trans,Total_Revenue,ecom_shopper,store_shopper,segment
227,13427900800000001,2020-11-17 08:02:21,2020-11-17 08:02:21,1,0.000000e+00,0,0.0,1,0.000000e+00,1,0,balance
299,7437010435000007,2020-11-16 14:54:42,2020-11-16 14:54:42,1,1.065814e-14,0,0.0,1,1.065814e-14,1,0,balance


## Tire Only

In [20]:
# Service & Tire & null
print(datetime.datetime.now())
query_tireonly_by_id="""
            select indiv_key, min(invoice_date) as min_date, max(invoice_date) as max_date, 
            count(distinct(case when store_id='ECOM' then trn_id end)) as Ecomm_Trans,
            sum(case when store_id='ECOM' then line_item_total_amt else 0 end) as Ecomm_Revenue,
            count(distinct(case when store_id!='ECOM' then trn_id end)) as Store_Trans,
            sum(case when store_id!='ECOM' then line_item_total_amt else 0 end) as Store_Revenue,
            count(distinct trn_id) as Total_Trans,
            sum(line_item_total_amt) as Total_Revenue
            from mdb.trn_detail
            where indiv_key is not null and invoice_date between %s and %s and product_type_cd='TIRE'
            group by indiv_key
            """  %(str_start_date,str_end_date)
print(query_tireonly_by_id)
df_tireonly_by_id=client.query(query_tireonly_by_id).result().to_dataframe()
print(df_tireonly_by_id.shape,df_tireonly_by_id['indiv_key'].nunique())
print(datetime.datetime.now())

2020-12-21 18:05:04.194302

            select indiv_key, min(invoice_date) as min_date, max(invoice_date) as max_date, 
            count(distinct(case when store_id='ECOM' then trn_id end)) as Ecomm_Trans,
            sum(case when store_id='ECOM' then line_item_total_amt else 0 end) as Ecomm_Revenue,
            count(distinct(case when store_id!='ECOM' then trn_id end)) as Store_Trans,
            sum(case when store_id!='ECOM' then line_item_total_amt else 0 end) as Store_Revenue,
            count(distinct trn_id) as Total_Trans,
            sum(line_item_total_amt) as Total_Revenue
            from mdb.trn_detail
            where indiv_key is not null and invoice_date between '2020-11-16' and '2020-12-02' and product_type_cd='TIRE'
            group by indiv_key
            


/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


(41635, 9) 41635
2020-12-21 18:05:09.766202


In [21]:
df_tireonly_by_id['ecom_shopper']=np.where(df_tireonly_by_id['Ecomm_Trans']>0,1,0)
df_tireonly_by_id['store_shopper']=np.where(df_tireonly_by_id['Ecomm_Trans']>0,0,1)

In [22]:
df_tireonly_by_id=pd.merge(df_tireonly_by_id,df_id_segment_label,on="indiv_key",how="left")
df_tireonly_by_id['segment']=df_tireonly_by_id['segment'].fillna("balance")

In [23]:
df_tireonly_by_id.head(2)

,indiv_key,min_date,max_date,Ecomm_Trans,Ecomm_Revenue,Store_Trans,Store_Revenue,Total_Trans,Total_Revenue,ecom_shopper,store_shopper,segment
0,13544621436000001,2020-11-25 15:18:14,2020-11-25 15:18:14,1,701.88,0,0.0,1,701.88,1,0,balance
1,13528850073000002,2020-11-24 10:53:55,2020-11-24 10:53:55,1,377.81,0,0.0,1,377.81,1,0,balance


In [24]:
print(df_tireonly_by_id['min_date'].min(),df_tireonly_by_id['min_date'].max())
print(df_tireonly_by_id['max_date'].min(),df_tireonly_by_id['max_date'].max())
# df_tireonly_by_id[df_tireonly_by_id['min_date']!=df_tireonly_by_id['max_date']]

2020-11-16 00:00:00 2020-12-02 00:00:00
2020-11-16 00:00:00 2020-12-02 00:00:00


In [25]:
func_agg={"indiv_key":"count","ecom_shopper":"sum","store_shopper":"sum",
          "Ecomm_Trans":"sum","Ecomm_Revenue":"sum",
          "Store_Trans":"sum","Store_Revenue":"sum",
          "Total_Trans":"sum","Total_Revenue":"sum",
          "min_date":"min","max_date":"max"
         }

df_output_audience_tireonly=df_tireonly_by_id.groupby('segment').agg(func_agg).reset_index()
df_output_audience_tireonly=df_output_audience_tireonly.rename(columns={"indiv_key":"shoppers"})
df_output_audience_tireonly=pd.merge(df_id_group_count,df_output_audience_tireonly,on="segment",how="outer")
df_output_audience_tireonly=df_output_audience_tireonly.sort_values("segment")

In [26]:
df_tireonly_by_id

,indiv_key,min_date,max_date,Ecomm_Trans,Ecomm_Revenue,Store_Trans,Store_Revenue,Total_Trans,Total_Revenue,ecom_shopper,store_shopper,segment
0,13544621436000001,2020-11-25 15:18:14,2020-11-25 15:18:14,1,701.88,0,0.00,1,701.88,1,0,balance
1,13528850073000002,2020-11-24 10:53:55,2020-11-24 10:53:55,1,377.81,0,0.00,1,377.81,1,0,balance
2,13413510542000001,2020-11-16 21:47:16,2020-11-16 21:47:16,1,490.07,0,0.00,1,490.07,1,0,balance
3,13499890833000001,2020-11-22 19:27:34,2020-11-22 19:27:34,1,1168.50,0,0.00,1,1168.50,1,0,balance
4,1222780775000827,2020-12-01 05:58:59,2020-12-01 05:58:59,1,377.89,0,0.00,1,377.89,1,0,balance
...,...,...,...,...,...,...,...,...,...,...,...,...
41630,13514483324000002,2020-11-23 00:00:00,2020-11-23 00:00:00,0,0.00,1,100.00,1,100.00,0,1,balance
41631,1222781702001736,2020-11-23 00:00:00,2020-11-23 00:00:00,0,0.00,1,688.43,1,688.43,0,1,balance
41632,13514482040000001,2020-11-23 00:00:00,2020-11-23 00:00:00,0,0.00,1,541.98,1,541.98,0,1,balance
41633,5450881511000003,2020-11-23 00:00:00,2020-11-23 00:00:00,0,0.00,1,88.99,1,88.99,0,1,balance


# Store Report

In [27]:
client.query("select distinct product_type_cd from mdb.trn_detail").result().to_dataframe()

/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,product_type_cd
0,TIRE
1,None
2,SERVICE


In [28]:
list_40_test_stores_biqquery=str(list_40_test_stores).replace("[","(").replace("]",")")
list_40_control_stores_biqquery=str(list_40_control_stores).replace("[","(").replace("]",")")
list_80_all_stores_biqquery=str(list_40_test_stores+list_40_control_stores).replace("[","(").replace("]",")")

## 2020 During Campaign

In [29]:
def pull_by_trans_with_date_range_80_stores(date_start_date,date_end_date):
    print(datetime.datetime.now())
    print(date_start_date,date_end_date)
    str_start_date="'"+str(date_start_date)+"'"
    str_end_date="'"+str(date_end_date)+"'"
    query_both_by_trans="""
                select trn_id, indiv_key, store_id, invoice_date, 
                sum(case when product_type_cd='TIRE' then line_item_total_amt else 0 end) as tire_sales,
                sum(case when product_type_cd='SERVICE' then line_item_total_amt else 0 end) as service_sales,
                sum(case when product_type_cd is null then line_item_total_amt else 0 end) as none_sales 
                from mdb.trn_detail
                where store_id in %s and invoice_date between %s and %s
                group by trn_id, indiv_key, store_id, invoice_date
                """  %(list_80_all_stores_biqquery,str_start_date,str_end_date)
    print(query_both_by_trans)
    df_both_by_trans=client.query(query_both_by_trans).result().to_dataframe()
    print(df_both_by_trans.shape,df_both_by_trans['store_id'].nunique())
    print(datetime.datetime.now())
    print(df_both_by_trans['invoice_date'].min(),df_both_by_trans['invoice_date'].max())
    
    print(df_both_by_trans['store_id'].apply(type).unique())
    print(df_both_by_trans['store_id'].apply(len).unique())
    df_both_by_trans['group']=np.where(df_both_by_trans['store_id'].isin(list_40_test_stores),"test","control")
    
    return df_both_by_trans



In [30]:
date_start_2020_during=datetime.datetime.strptime(str_start_date.replace("'",""),"%Y-%m-%d").date()
date_end_2020_during=datetime.datetime.strptime(str_end_date.replace("'",""),"%Y-%m-%d").date()

date_start_2020_pre=date_start_2020_during-datetime.timedelta(days=21)
date_end_2020_pre=date_end_2020_during-datetime.timedelta(days=21)


date_start_2019_during=date_start_2020_during-datetime.timedelta(days=52*7)
date_end_2019_during=date_end_2020_during-datetime.timedelta(days=52*7)

date_start_2019_pre=date_start_2019_during-datetime.timedelta(days=21)
date_end_2019_pre=date_end_2019_during-datetime.timedelta(days=21)


In [31]:
df_both_by_trans_during_2020=pull_by_trans_with_date_range_80_stores(date_start_2020_during,date_end_2020_during)
print("\n")
df_both_by_trans_pre_2020=pull_by_trans_with_date_range_80_stores(date_start_2020_pre,date_end_2020_pre)
print("\n")
df_both_by_trans_during_2019=pull_by_trans_with_date_range_80_stores(date_start_2019_during,date_end_2019_during)
print("\n")
df_both_by_trans_pre_2019=pull_by_trans_with_date_range_80_stores(date_start_2019_pre,date_end_2019_pre)
print("\n")

2020-12-21 18:05:11.643175
2020-11-16 2020-12-02

                select trn_id, indiv_key, store_id, invoice_date, 
                sum(case when product_type_cd='TIRE' then line_item_total_amt else 0 end) as tire_sales,
                sum(case when product_type_cd='SERVICE' then line_item_total_amt else 0 end) as service_sales,
                sum(case when product_type_cd is null then line_item_total_amt else 0 end) as none_sales 
                from mdb.trn_detail
                where store_id in ('2728', '2178', '2842', '4128', '6227', '0431', '2677', '2750', '4246', '6658', '2724', '2733', '6626', '2248', '4739', '2621', '4328', '8963', '2844', '4738', '2766', '2759', '1224', '2180', '4722', '6258', '4730', '2251', '2777', '6257', '2249', '6568', '6329', '2928', '6325', '4736', '2761', '2680', '8426', '9124', '6661', '4248', '8259', '2523', '0231', '2325', '2654', '0845', '4733', '8836', '6552', '8152', '4330', '2476', '4230', '6641', '2647', '8650', '2424', '2381', '0933', '0

/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


(16612, 7) 80
2020-12-21 18:05:14.844952
2020-11-16 00:00:00 2020-12-02 00:00:00
[<class 'str'>]
[4]


2020-12-21 18:05:14.857849
2020-10-26 2020-11-11

                select trn_id, indiv_key, store_id, invoice_date, 
                sum(case when product_type_cd='TIRE' then line_item_total_amt else 0 end) as tire_sales,
                sum(case when product_type_cd='SERVICE' then line_item_total_amt else 0 end) as service_sales,
                sum(case when product_type_cd is null then line_item_total_amt else 0 end) as none_sales 
                from mdb.trn_detail
                where store_id in ('2728', '2178', '2842', '4128', '6227', '0431', '2677', '2750', '4246', '6658', '2724', '2733', '6626', '2248', '4739', '2621', '4328', '8963', '2844', '4738', '2766', '2759', '1224', '2180', '4722', '6258', '4730', '2251', '2777', '6257', '2249', '6568', '6329', '2928', '6325', '4736', '2761', '2680', '8426', '9124', '6661', '4248', '8259', '2523', '0231', '2325', '2654', '0845', '47

In [32]:
df_both_by_trans_during_2020['period']="during_2020"
df_both_by_trans_pre_2020['period']="pre_2020"
df_both_by_trans_during_2019['period']="during_2019"
df_both_by_trans_pre_2019['period']="pre_2019"


In [33]:
df_both_by_trans=pd.concat([df_both_by_trans_during_2020,df_both_by_trans_pre_2020,df_both_by_trans_during_2019,df_both_by_trans_pre_2019])
df_both_by_trans.shape

(76134, 9)

In [34]:
df_both_by_trans['tire_trans']=np.where(df_both_by_trans['tire_sales']>0,1,0)
df_both_by_trans['service_trans']=np.where(df_both_by_trans['service_sales']>0,1,0)
df_both_by_trans['other_trans']=np.where(df_both_by_trans['tire_trans']==1,0,
                                         np.where(df_both_by_trans['service_sales']==1,0,1)
                                        )

In [35]:
func_agg={"indiv_key":"nunique",
          "tire_sales":"sum","service_sales":"sum","none_sales":"sum",
          "tire_trans":"sum","service_trans":"sum","other_trans":"sum",
          "invoice_date":"min","invoice_date":"max"
         }

In [36]:
df_output_by_store_period=df_both_by_trans.groupby(["period","group","store_id"]).agg(func_agg).reset_index()
df_output_by_store_period=df_output_by_store_period.rename(columns={"indiv_key":"total_unique_shopper"})

In [37]:
# Per the disscussion, the shopper count per type is by each store in the same period per id, not per transaction 
df_shoppers_by_type=pd.DataFrame()
for period,df_group in df_both_by_trans.groupby("period"):
    df_tire=df_group[df_group['tire_sales']>0]
    df_tire=df_tire[['period','group','store_id','indiv_key']].drop_duplicates()
    df_tire['tire_shopper']=1
    
    df_service=df_group[df_group['service_sales']>0]
    df_service=df_service[['period','group','store_id','indiv_key']].drop_duplicates()
    df_service=pd.merge(df_service,df_tire,on=['period','group','store_id','indiv_key'],how="left")
    df_service=df_service[df_service['tire_shopper']!=1]
    del df_service['tire_shopper']
    df_service['service_shopper']=1

    
    df_tire_count=df_tire.groupby(['period','group','store_id'])['tire_shopper'].sum().to_frame().reset_index()
    df_service_count=df_service.groupby(['period','group','store_id'])['service_shopper'].sum().to_frame().reset_index()
    df_shopper_count=pd.merge(df_tire_count,df_service_count,on=['period','group','store_id'],how="outer")
    df_shopper_count=df_shopper_count.fillna(0)
    df_shoppers_by_type=df_shoppers_by_type.append(df_shopper_count)
df_output_by_store_period=pd.merge(df_output_by_store_period,df_shoppers_by_type,on=["period","group","store_id"],how="left")


In [38]:
df_output_by_store_period

,period,group,store_id,total_unique_shopper,tire_sales,service_sales,none_sales,tire_trans,service_trans,other_trans,invoice_date,tire_shopper,service_shopper
0,during_2019,control,0231,138,12734.13,19085.32,0.0,34,95,115,2019-12-04,34,56
1,during_2019,control,0461,347,46654.05,50474.72,0.0,113,348,337,2019-12-04,91,203
2,during_2019,control,0466,178,31053.07,15931.64,0.0,59,164,145,2019-12-04,58,93
3,during_2019,control,0544,285,48070.20,21016.81,0.0,90,216,221,2019-12-04,88,119
4,during_2019,control,0845,202,27784.04,27787.12,0.0,68,188,150,2019-12-04,67,107
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,pre_2020,test,6626,208,27953.03,33253.43,0.0,62,183,177,2020-11-11,58,107
316,pre_2020,test,6658,177,21808.68,39457.86,0.0,45,160,149,2020-11-11,43,107
317,pre_2020,test,8426,216,39468.23,44868.76,0.0,66,204,193,2020-11-11,64,113
318,pre_2020,test,8963,208,24175.96,20618.04,0.0,64,179,167,2020-11-11,63,107


In [39]:
# QC below

df=df_both_by_trans[df_both_by_trans['period']=="during_2019"]
df=df[df['store_id']=="0231"]
df.shape

(149, 12)

In [40]:
df

,trn_id,indiv_key,store_id,invoice_date,tire_sales,service_sales,none_sales,group,period,tire_trans,service_trans,other_trans
0,191892201911180231,1.222781e+15,0231,2019-11-18,0.00,0.00,0.0,control,during_2019,0,0,1
1,191952201911250231,1.222781e+15,0231,2019-11-25,0.00,1149.48,0.0,control,during_2019,0,1,1
2,192004201911290231,8.330131e+15,0231,2019-11-29,0.00,0.00,0.0,control,during_2019,0,0,1
3,191942201911220231,8.229531e+15,0231,2019-11-22,0.00,240.24,0.0,control,during_2019,0,1,1
4,191918201911210231,1.222780e+15,0231,2019-11-21,0.00,1526.80,0.0,control,during_2019,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
144,192043201912040231,1.222780e+15,0231,2019-12-04,273.98,148.90,0.0,control,during_2019,1,1,0
145,191922201911210231,1.222782e+15,0231,2019-11-21,0.00,0.00,0.0,control,during_2019,0,0,1
146,191927201911230231,1.222782e+15,0231,2019-11-23,0.00,88.00,0.0,control,during_2019,0,1,1
147,191980201911260231,8.243712e+15,0231,2019-11-26,0.00,65.00,0.0,control,during_2019,0,1,1


In [41]:
df['indiv_key'].nunique()

138

In [42]:
df_qc=df[(df['tire_sales']<=0) & (df['service_sales']<=0)]
df_qc['indiv_key'][0]

1222781397000741.0

In [43]:
df_qc.head(2)

,trn_id,indiv_key,store_id,invoice_date,tire_sales,service_sales,none_sales,group,period,tire_trans,service_trans,other_trans
0,191892201911180231,1.222781e+15,0231,2019-11-18,0.0,0.0,0.0,control,during_2019,0,0,1
2,192004201911290231,8.330131e+15,0231,2019-11-29,0.0,0.0,0.0,control,during_2019,0,0,1


In [44]:
df_both_by_trans[df_both_by_trans['trn_id']=="191892201911180231"]

# To show YG

,trn_id,indiv_key,store_id,invoice_date,tire_sales,service_sales,none_sales,group,period,tire_trans,service_trans,other_trans
0,191892201911180231,1.222781e+15,0231,2019-11-18,0.0,0.0,0.0,control,during_2019,0,0,1


In [45]:
writer=pd.ExcelWriter("./GY_2020_Nov_Pilot_campaign_performance_JL_%s.xlsx"%str(datetime.datetime.now().date()),engine="xlsxwriter")
df_output_audience_both.to_excel(writer,"audience_by_group_both",index=False)
df_output_audience_tireonly.to_excel(writer,"audience_by_group_tireonly",index=False)

df_both_by_trans.to_excel(writer,"trans_detail",index=False)
df_output_by_store_period.to_excel(writer,"store_by_each_period",index=False)

writer.save()

In [46]:
df_both_by_trans.shape

(76134, 12)